In [1]:
cd ..

/mnt/WD Blue/Machine Learning/Proyectos/Chatbot-en-Espanol


In [2]:
import sentencepiece as spm
from settings import *
import pickle

In [3]:
def load_lines(how_many):
    n = 0
    with open(corpus, 'rb') as readfl:
        for i, line in enumerate(readfl):
            if max(line.find(b"["),line.find(b"]")) == -1: #filtra los comentarios
                n+=1
                line = line.lstrip(b"- ").decode()
                yield i,line
                if n==total_lines: break

In [4]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
total_lines=500_000

with open(f"{save_dir}/writefile.txt", 'w') as writefl:
    for i,line in load_lines(total_lines):
        writefl.write(line)

In [5]:
cmd = (f'--input={save_dir}/writefile.txt --vocab_size=15000 --model_prefix=m_bpe --model_type=bpe '
       f'--pad_id={PAD_token} --unk_id={UNK_token} --bos_id={SOS_token} --eos_id={EOS_token} --pad_piece=[PAD] '
       f'--unk_piece=[UNK] --bos_piece=[SOS] --eos_piece=[EOS] --normalization_rule_name=nfkc_cf')

spm.SentencePieceTrainer.train(cmd)
os.rename('./m_bpe.vocab',f'./{save_dir}/m_bpe.vocab')
os.rename('./m_bpe.model',f'./{save_dir}/m_bpe.model')

sp = spm.SentencePieceProcessor()
sp.load(f'{save_dir}/m_bpe.model')

True

In [6]:
for id in range(4):
    print(sp.id_to_piece(id), sp.is_control(id))

print('bos=', sp.bos_id(),', eos=', sp.eos_id(),', unk=', sp.unk_id(),', pad=', sp.pad_id())

# print(sp.encode_as_ids('Hello world'))
# print([sp.bos_id()] + sp.encode_as_ids('H O L A world') + [sp.eos_id()])

# print(sp.encode_as_pieces('Hello world'))
# print(sp.encode_as_pieces('HOLA Hola hola mundo estás')) 
# print([sp.bos_id()] + sp.encode_as_ids('Hola mundo estás') + [sp.eos_id()])

[PAD] True
[SOS] True
[EOS] True
[UNK] False
bos= 1 , eos= 2 , unk= 3 , pad= 0


In [7]:
def gen_pairs(how_many): #how many loaded line, pairs will be less
    prev_ln = None
    for i, line in load_lines(how_many):
        if prev_ln == None: 
            prev_ln = sp.encode_as_ids(line.strip()) + [EOS_token]
            prev_idx= i
            continue
        
        line = sp.encode_as_ids(line.strip()) + [EOS_token]
        if prev_idx+1 == i:
            yield [prev_ln,line]

        if i == how_many: break
        prev_ln = line
        prev_idx= i

In [8]:
# for pair in gen_pairs(100):
#     print(pair[0],sp.encode_as_pieces(sp.decode_ids(pair[0])),sp.decode_ids(pair[0]),sep="\n")
#     print(pair[1],sp.encode_as_pieces(sp.decode_ids(pair[1])),sp.decode_ids(pair[1]),sep="\n")
#     print()

In [9]:
pairs = list(gen_pairs(LINES_USED))

In [10]:
with open(save_dir + '/sp_pairs.pkl', 'wb') as file:
    pickle.dump(pairs, file)